<a href="https://colab.research.google.com/github/roquearmas/IABD_DATA/blob/main/Predicci%C3%B3n_y_salida_de_Reconocimiento_Acordes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import librosa
import librosa.display

import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from google.colab import drive
import itertools
import math
from IPython.display import Audio
import math

import tempfile
import os
import keras.models

import cv2
from PIL import Image


Librosa(https://zenodo.org/badge/6309729.svg)


Carga del fichero de audio a predecir

In [2]:
drive.mount('/content/drive')
audio_file = '/content/drive/MyDrive/proyecto deteccion acordes/Dover - King George Acústico 2012.mp3'
# audio_file = '/content/drive/MyDrive/proyecto deteccion acordes/02 - Eric Clapton - Before You Accuse Me .mp3'
#audio_file = '/content/drive/MyDrive/proyecto deteccion acordes/Lynyrd Skynyrd - 01 - Sweet Home Alabama.mp3'
# Carga en la librería Librosa
audio_data, sample_rate = librosa.load(audio_file)



Mounted at /content/drive


Carga del modelo preentrenado

In [3]:
model = keras.models.load_model('/content/drive/MyDrive/proyecto deteccion acordes/modelo_entrenado/recon_acordes.h5')


Array de acordes soportados, en el mismo orden en que se entrenó el modelo

In [4]:
lista_acordes= ['Am', 'Bb', 'C', 'Dm', 'Bdim', 'G', 'F', 'Em']
acordes =np.array(lista_acordes)

**Tratamiento del fichero de audio**:

1.   Extraer los datos del fichero de audio, de la misma manera que cuando se entrenó el modelo

    -   División del fichero en muestras de medio segundo
    -   Normalización de la muestra a imagen en blanco y negro
    -   Reescalado de la imagen a 300 x 300 píxeles

2.   Predecir cada muestra de tiempo de la canción
3.   Generar un fichero de salida con el tiempo de inicio y fin de la muestra y el acorde predicho


In [40]:
# Divide el audio en segmentos de 1/2 segundo y genera histogramas y periodogramas
segment_duration = 0.5  # Duración del segmento en segundos
num_segments = int(len(audio_data) / (sample_rate * segment_duration))
#separar la parte armónica de la percusiva del audio de origen
audio_harmonic, audio_percussive = librosa.effects.hpss(audio_data)
salida = []
for i in range(num_segments):

    start_time = i * segment_duration
    end_time = (i + 1) * segment_duration
    segment = audio_harmonic[int(start_time * sample_rate):int(end_time * sample_rate)]

    # Deshabilitar la visualización interactiva de las imágenes
    plt.ioff()

    # Periodograma del segmento
    plt.figure(figsize=(4, 4))
    librosa.display.specshow(librosa.power_to_db(abs(librosa.stft(segment)), ref=np.max), y_axis='log', x_axis='time', cmap='viridis')
    plt.axis('off')
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
     # Guardar la imagen
    # nombreimagen = os.path.join('/content/', 'imagen.png')
    # plt.savefig(nombreimagen, bbox_inches='tight', pad_inches=0)
    # Convertir la figura a un array de NumPy
    fig = plt.gcf()
    fig.canvas.draw()
    espectrograma_array = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    espectrograma_array = espectrograma_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    # Cerrar la figura de Matplotlib
    plt.close()
    imagen=normalizar_imagen(espectrograma_array)
    # normalizar_imagen('/content/')
    #cargar imagen en X
    X = []
    X.append(imagen)
      #hacer predict con el modelo
    pred = model.predict(np.array(X))
    #añadir linea con el predict a un fichero de texto
    row = []
    row.append(start_time) # inicio del segmento a estudiar
    row.append(end_time)   # fin del segmento
    predicho = np.argmax(pred) #clase con mejor predicción
    row.append(acordes[predicho]) #acorde que pertenece a esa clase
    salida.append (row) #añado la fila al array de la canción
    print(row)

# Escribir los datos en el archivo de texto
with open('/content/salida.srt', 'w') as file:
    for row in salida:
        row_str = ' '.join(map(str, row))  # Convertir la fila a una cadena de texto separada por espacios
        file.write(row_str + '\n')  # Escribir la fila en el archivo



(300, 300, 3)
1/1 [==============================] - 0s 103ms/step
[[7.9649502e-01 3.0405917e-07 4.4467433e-06 1.1389869e-03 1.2904064e-09
  2.3559888e-07 1.6940279e-02 1.8542081e-01]]
0
[0.0, 0.5, 'Am']
(300, 300, 3)
1/1 [==============================] - 0s 92ms/step
[[2.0095186e-01 2.3699612e-03 1.8469911e-03 2.3297425e-01 9.6393712e-02
  4.6355417e-01 3.0436667e-04 1.6047343e-03]]
5
[0.5, 1.0, 'G']
(300, 300, 3)
1/1 [==============================] - 0s 94ms/step
[[0.02499853 0.01555419 0.31705853 0.04411837 0.00229059 0.4445141
  0.00044767 0.15101801]]
5
[1.0, 1.5, 'G']
(300, 300, 3)
1/1 [==============================] - 0s 96ms/step
[[0.24717872 0.00450354 0.13930787 0.14234392 0.00984986 0.40218487
  0.01481942 0.03981182]]
5
[1.5, 2.0, 'G']
(300, 300, 3)
1/1 [==============================] - 0s 94ms/step
[[0.03245145 0.00307494 0.4725141  0.0560842  0.00999836 0.34031108
  0.00092131 0.08464457]]
2
[2.0, 2.5, 'C']
(300, 300, 3)
1/1 [==============================] - 0s 87ms/

In [35]:
X.shape

AttributeError: 'list' object has no attribute 'shape'

Paso la imagen a escala de grises, antes de evaluarla

In [33]:
def normalizar_imagen(espectrograma):
  # files = os.listdir(os.path.join(dirname))
  # images = [file for file in files if file.lower().endswith("png")]
  # for image_name in images:
  #   imagen = cv2.imread(os.path.join(dirname, image_name))
  #   # Convierto todas las imágenes a escala de grises
  #   imagen_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY) # Normalizo la imagen a escala de grises
  #   imagen_gris = Image.fromarray(imagen_gris)
  #   imagen_gris.save(os.path.join( dirname, image_name))

  #  imagen_gris = cv2.cvtColor(espectrograma, cv2.COLOR_BGR2GRAY) # Normalizo la imagen a escala de grises

  # Convertir a escala de grises
  imagen_gris = cv2.cvtColor(espectrograma, cv2.COLOR_BGR2GRAY)
  espectrograma_gris_3_canales = cv2.cvtColor(imagen_gris, cv2.COLOR_GRAY2BGR)
  imagen = cv2.resize(espectrograma_gris_3_canales, (300,300), interpolation=cv2.INTER_CUBIC)
  # imagen_gris = Image.fromarray(imagen_gris)
  return imagen


In [22]:
imagen_gris.shape

NameError: name 'imagen_gris' is not defined